In [36]:
import json
import prettytable
from neo4j import GraphDatabase
from dotenv import load_dotenv
from gen_query import handle_gen_query
import util
from get_entities import handle_get_entities

load_dotenv()


False

In [41]:
# Connect neo4j DB.
driver = GraphDatabase.driver('bolt://localhost:7687',
                              auth=('neo4j', 'password'))


In [42]:
def gen_query_ontology(text):
    table = prettytable.PrettyTable(["Step", "Result"])
    table.max_width = 80
    table.add_row(["Input", text])

    concepts = util.get_concepts(text)

    match_relation = util.get_match_relation(concepts.keys())

    evidences = handle_get_entities(text)

    table.add_rows([["Match alias", concepts], ["Find individuals",
                                                evidences]])

    targets = list(set(concepts.keys()).difference(set(evidences.keys())))
    table.add_row(["Target concepts", targets])

    query = handle_gen_query(concepts.keys(), evidences, match_relation)
    table.add_row(["Query", query])

    print(table)
    # print(query)
    return query

In [43]:
def run_query(query):
    with driver.session(database="htdb") as session:
        results = session.run(query)

        table_results = prettytable.PrettyTable(results.keys())
        table_results.max_width = 60
        for r in results:
            table_results.add_row(r.values())

        print(table_results)
        # return table_results

In [60]:
with open('data/question_dict.json', 'r', encoding="utf-8") as fp:
    question_dict = json.load(fp)
    fp.close()

REAL_ESTATE_TYPE = question_dict["real_estate_type"]
PRICE = question_dict["price"]
AREA = question_dict["area"]
DISTRICT = question_dict["district"]
BED_ROOM = question_dict["bed_room"]
FLOOR = question_dict["floor"]
LEGAL = question_dict["legal"]
POSITION = question_dict["position"]
LOCATION = question_dict["location"]
POTENTIAL = question_dict["potential"]

In [61]:
for question in DISTRICT:
    cqlNodeQuery = gen_query_ontology(question)
    run_query(cqlNodeQuery)
    print(
        "----------------##########################################----------------"
    )


+------------------+----------------------------------------------------------------------------------+
|       Step       |                                      Result                                      |
+------------------+----------------------------------------------------------------------------------+
|      Input       |                nhà hoặc căn hộ giá khoảng 2 tỷ thì mua ở quận nào                |
|   Match alias    | {'real_estate_type': 'căn hộ', 'transaction': 'mua', 'price': 'giá', 'district': |
|                  |                                  'ở quận nào'}                                   |
| Find individuals |     {'real_estate_type': ['nha', 'can ho'], 'price': [(2000000000.0, None)],     |
|                  |                             'transaction': ['mua']}                              |
| Target concepts  |                                   ['district']                                   |
|      Query       |                                            